
Строим запрос к нашему сервису

In [1]:
import requests

import urllib.request
import json 

In [2]:
# Пример данных
data = ( 
    "Export GIT LOG into an Excel file",
    "I have looked into the forum, but with"
)

In [3]:
def send_json(x):
    title_, body_ = x
    print(x)
    body = {
        'Title': title_, 
        'Body': body_
        }
    myurl = 'http://7c02-35-233-158-247.ngrok-free.app' + '/predict'
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=body, headers=headers)
    return response.json()['predictions']

In [4]:
# обращение к серверу с запросом из одного набора (его построили руками выше - data)
response = send_json(data)
print('предсказание', response)

('Export GIT LOG into an Excel file', 'I have looked into the forum, but with')
предсказание 0.689907026637619


Сделаем обработку массы запросов:

1. загрузим данные из valid.csv

In [5]:
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score
from urllib import request, parse


In [6]:
df = pd.read_csv("./valid.csv")
df.head(5)

,Id,Title,Body,Tags,CreationDate,Y
0,34552974,How to get all the child records from differen...,I am having 4 different tables like \r\nselect...,<sql><sql-server>,2016-01-01 01:44:52,LQ_EDIT
1,34554721,Retrieve all except some data of the another t...,I have two table m_master and tbl_appointment\...,<php><mysql><sql><codeigniter><mysqli>,2016-01-01 08:43:50,LQ_EDIT
2,34555135,Pandas: read_html,<p>I'm trying to extract US states from wiki U...,<python><pandas>,2016-01-01 09:55:22,HQ
3,34555448,Reader Always gimme NULL,"I'm so new to C#, I wanna make an application ...",<sql-server><c#-4.0>,2016-01-01 10:43:45,LQ_EDIT
4,34555752,php rearrange array elements based on condition,basically i have this array:\r\n\r\n array(...,<php>,2016-01-01 11:34:09,LQ_EDIT


In [7]:
df['Y'] = df['Y'].apply(lambda x: 0 if x == 'HQ' else 1)

2. Передаем запрос одиночный из наших данных и посмотрим на ответ

In [8]:
response = send_json(df[['Title', 'Body']].iloc[0,:])
print('предсказание', response)

Title    How to get all the child records from differen...
Body     I am having 4 different tables like \r\nselect...
Name: 0, dtype: object
предсказание 0.9396746872872297


3. Сделаем N запросов и оценим время

In [9]:
N = 25

In [10]:
%%time
predictions = df[
                     ['Title', 'Body']
                     ].iloc[:N].apply(lambda x: send_json(x), axis=1)

Title    How to get all the child records from differen...
Body     I am having 4 different tables like \r\nselect...
Name: 0, dtype: object
Title    Retrieve all except some data of the another t...
Body     I have two table m_master and tbl_appointment\...
Name: 1, dtype: object
Title                                    Pandas: read_html
Body     <p>I'm trying to extract US states from wiki U...
Name: 2, dtype: object
Title                             Reader Always gimme NULL
Body     I'm so new to C#, I wanna make an application ...
Name: 3, dtype: object
Title      php rearrange array elements based on condition
Body     basically i have this array:\r\n\r\n    array(...
Name: 4, dtype: object
Title     How do I make a constructor for a derived class?
Body     <p>I am trying to make a constructor for a der...
Name: 5, dtype: object
Title    how can i create a dynamic tow dimensional arr...
Body      I am using c++ in my lesson and for solving a...
Name: 6, dtype: object
Title        

In [11]:
predictions.values[:5]

array([0.93967469, 0.99152112, 0.01292043, 0.93363839, 0.99376318])

4. Посчитаем метрику

In [12]:
y_test = df['Y']

In [13]:
import numpy as np

precision, recall, thresholds = precision_recall_curve(y_test[:N], predictions)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.6131603263981417, F-Score=0.970, Precision=0.941, Recall=1.000


In [14]:
roc_auc_score(y_score=predictions.values, y_true=y_test.values[:N])

0.9930555555555555